In [3]:
import sys
assert sys.version_info >= (3, 5)

In [4]:
import sklearn
assert sklearn.__version__ >= "0.20"

In [5]:
import numpy as np
import os
import tarfile
import urllib
import pandas as pd
import urllib.request

In [6]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt

Get the CSV data in to a panda dataframe, might need to alter filepaths depending where this is saved

In [234]:
def load_data():
    return pd.read_csv("data/winemag-data-130k-v2.csv")

wine = load_data()
wine.head()

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


Ideas for pre-processing:

Create Vintage feature by scraping data from title field

Maybe scrape description for descriptive keywords?

Maybe try to find the longitude and latitude of the winery for some scattergraph data


Remove taster_twitter_handle

Remove Unnamed id feature

Drop the unneeded Id and taster twitter handle columns

In [231]:
wine = wine.drop(['Unnamed: 0', 'taster_twitter_handle'], axis=1)

Create a new feature for the vintage of the wine, get the data from the title field

Use RegEx to find the vintage year, make sure year is between 1975 and 2022 to avoid incorrect data. Some names don't contain vintage year, might need to remove those

In [8]:
import re

In [215]:
def getVintage(wine):
    f = re.findall(r'\d\d\d\d', wine["title"])
    if len(f) == 0:
        return int()
    elif len(f) > 1:
        for y in f:
            if int(y) < 1975 and int(y) > 2022:
                del f[y]
    f = f[0]
    return f

In [217]:
wine["vintage"] = wine.apply(getVintage, axis=1)

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,taster_name,taster_twitter_handle,title,variety,winery,vintage
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,NaN,Sicily & Sardinia,Etna,NaN,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia,2013
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.0,Douro,NaN,NaN,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos,2011
2,2,US,"Tart and snappy, the flavors of lime flesh and...",NaN,87,14.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm,2013
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.0,Michigan,Lake Michigan Shore,NaN,Alexander Peartree,NaN,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian,2013
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.0,Oregon,Willamette Valley,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129966,129966,Germany,Notes of honeysuckle and cantaloupe sweeten th...,Brauneberger Juffer-Sonnenuhr Spätlese,90,28.0,Mosel,NaN,NaN,Anna Lee C. Iijima,NaN,Dr. H. Thanisch (Erben Müller-Burggraef) 2013 ...,Riesling,Dr. H. Thanisch (Erben Müller-Burggraef),2013
129967,129967,US,Citation is given as much as a decade of bottl...,NaN,90,75.0,Oregon,Oregon,Oregon Other,Paul Gregutt,@paulgwine,Citation 2004 Pinot Noir (Oregon),Pinot Noir,Citation,2004
129968,129968,France,Well-drained gravel soil gives this wine its c...,Kritt,90,30.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Gresser 2013 Kritt Gewurztraminer (Als...,Gewürztraminer,Domaine Gresser,2013
129969,129969,France,"A dry style of Pinot Gris, this is crisp with ...",NaN,90,32.0,Alsace,Alsace,NaN,Roger Voss,@vossroger,Domaine Marcel Deiss 2012 Pinot Gris (Alsace),Pinot Gris,Domaine Marcel Deiss,2012


Drop all records that have no indication of location

In [232]:
locCols = ["country", "province", "region_1"]
wine = wine.dropna(axis=0,subset=locCols, how='all', inplace=True)

This is a script that tries to get the longitude and latitude of each wineries location, it queries an online google maps esque webpage and gradually widens the search area until there is a match. It is not viable to get all of the data this way.

In [1]:
%pip install requests

  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached certifi-2022.9.14-py3-none-any.whl (162 kB)
Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import urllib.parse

In [135]:
def getCoords(wine):
    address = str(wine["winery"]) + "," + str(wine["province"]) + "," + str(wine["region_1"]) + "," + str(wine["country"])
    repCount = 0
    while True:
        url = 'https://nominatim.openstreetmap.org/search/' + urllib.parse.quote(address) +'?format=json'
        response = requests.get(url).json()
        if response:
            print(response[0]["lat"])
            print(response[0]["lon"])
            return [response[0]["lat"], response[0]["lon"]]
        address = address.split(',')
        address = ','.join(address[1:])

If we let the code run, we should create some code to populate the longitude and latitude of the other records of each winery. At this current point, only the first record of each winery has their long and lat updated

In [136]:
# Get the first record of each unique winery
uniqueWineries = wine.groupby("winery", as_index=False).first()

uniqueWineries["longitude"] = uniqueWineries.apply(getCoords, axis=1)[0]
uniqueWineries["latitude"] = uniqueWineries.apply(getCoords, axis=1)[1]

42.31658175
1.6111496867672197
35.638990199999995
-120.6860902269523
37.6090291
-121.899142
38.9347122
-123.72009793467652
35.531957
-81.985687
-33.3107798
151.4350326
41.6591121
-3.6815871170201557
35.531957
-81.985687
-41.5000831
172.8344077
46.211056
-119.7465957
38.47023425
-123.15474461850283
43.436678799999996
6.654233406805152
45.3676909
-123.00640669254446
38.13656605102041
-121.28039039795918
45.9979808
-118.3989894
30.8124247
34.8594762
38.4019643
-120.813261
-35.1481469
139.2631777
37.7610093
-25.6620881
-34.6447542
-58.62232627477273
44.9622232
-85.485356
-35.0055092
138.6329958
38.8983696
-77.0058628
33.0913463
-117.3100772
42.3055058
-78.6927027
-30.7054363
-69.1988222
33.9304473
-117.2019307
42.389022
-122.8653263
38.4019643
-120.813261
34.947792
-120.564712
32.5419837
-117.0772328
40.48714763561227
-122.29882882486017
38.88898935
-77.04022726522987
-34.5957331
139.003646
33.9304473
-117.2019307
34.5881974
-117.1801446
-41.5000831
172.8344077
-25.7590337
28.2547601610210

KeyboardInterrupt: 